# Personal RAG

In [ ]:
#import locale
#def getpreferredencoding(do_setlocale = True):
#    return "UTF-8"
#locale.getpreferredencoding = getpreferredencoding

I use llama-index==0.9.42 because in the newer version, the similarity yields incorrect results. They likely switched the scoring from 1-score to score, resulting in no nodes being retrieved anymore.

In [ ]:
#!pip install peft
#!pip install bitsandbytes
#!pip install accelerate
#!pip install llama-index==0.9.42
#!pip install pyvis
#!pip install rank_bm25
#!pip install langdetect
#!pip install gradio

## Data

In [ ]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch
from llama_index.llms import HuggingFaceLLM
from typing import List, Tuple
from itertools import product
from nltk.tokenize import word_tokenize
import pandas as pd
import os
import glob
import numpy as np
from langdetect import detect
import nltk
from tqdm import tqdm
import json
from llama_index.schema import Document
from llama_index.node_parser import SentenceSplitter, get_leaf_nodes, get_root_nodes
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.storage import StorageContext
from llama_index import load_index_from_storage, VectorStoreIndex, ServiceContext
from llama_index.prompts import PromptTemplate
from llama_index.prompts.base import PromptTemplate
from llama_index.prompts.prompt_type import PromptType
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.response_synthesizers import ResponseMode
from llama_index.retrievers import BM25Retriever
from llama_index.retrievers import QueryFusionRetriever
from llama_index.response_synthesizers import SimpleSummarize
from llama_index.response_synthesizers import Refine
from llama_index.response_synthesizers import TreeSummarize
from llama_index.postprocessor import SimilarityPostprocessor
from llama_index.postprocessor import SentenceEmbeddingOptimizer
from llama_index.postprocessor import LLMRerank
from llama_index.schema import  NodeWithScore, QueryBundle
from typing import List, Tuple
from llama_index.response_synthesizers.type import ResponseMode
from llama_index.response_synthesizers import get_response_synthesizer
from llama_index.vector_stores.types import MetadataFilters,MetadataFilter, ExactMatchFilter, FilterCondition, FilterOperator
import gradio as gr

In [ ]:
path_project = os.path.join(os.getcwd())

In [ ]:
prompt_str = "Can you generate only one question about the following topic: {topic}. Reply with the question only"
prompt_tmpl = PromptTemplate(prompt_str)

prompt_tmpl

In [ ]:
def generate_prompt(prompt_str, topic):
    prompt_tmpl = PromptTemplate(prompt_str)
    return prompt_tmpl.format(topic=topic)

In [ ]:
test = generate_prompt(prompt_str, 'Chinese international trade')
test

In [ ]:
from nltk.tokenize import word_tokenize

def count_tokens(text):
    tokens = word_tokenize(text)
    return len(tokens)

Data below is used to populate the dropdown menu in the tags

In [ ]:
path_notes = os.path.join(path_project, "XXX")
# Open and read the JSON file
with open(path_notes, 'r') as file:
    data = json.load(file)

df = (
    pd.DataFrame(data)
    .drop_duplicates(subset = ['title'])
    .assign(
        size_doc = lambda x: x['markdown'].apply(count_tokens),
        title_size = lambda x: x['title'].apply(count_tokens)
    )
    .loc[lambda x: x['title_size']< 30]
    .assign(date = lambda x: pd.to_datetime(x['created']).dt.strftime('%Y-%m-%d'))
    .reset_index(drop = True)
)


In [ ]:
keyword = list(df.explode('tags')['tags'].sort_values().unique())
date = list(df.explode('date')['date'].sort_values(ascending = False).unique())

## Gradio

### Load model

In [ ]:
model_llm = "mistralai/Mistral-7B-Instruct-v0.2"

In [ ]:
torch.set_default_device('cuda')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model_base = AutoModelForCausalLM.from_pretrained(
     model_llm,
    quantization_config=bnb_config,
    device_map={"": 0})

system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."
# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = "<|USER|>{query_str}<|ASSISTANT|>"
tokenizer = AutoTokenizer.from_pretrained(model_llm)
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=500,
    generate_kwargs={"temperature": 0.3, "do_sample": True},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer=tokenizer,
    model=model_base,
    device_map="auto",
    tokenizer_kwargs={"max_length": 4096},
    model_kwargs={"torch_dtype": torch.float16}

)

In [ ]:
from llama_index.embeddings import HuggingFaceEmbedding
model = "sentence-transformers/all-MiniLM-L6-v2"
embed_model = HuggingFaceEmbedding(model_name=model)
text_splitter = SentenceSplitter(chunk_size=600, chunk_overlap=90)
### with
service_context_with_splitter = ServiceContext.from_defaults(
        embed_model=embed_model,
        llm=llm,
        text_splitter=text_splitter
    )

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir=os.path.join(path_project, "PERSISTENT_STORAGE_PKM"))

# load index
index_with_splitter = load_index_from_storage(storage_context,
                                service_context=service_context_with_splitter)

### Load vector store

In [ ]:
folder_vector = 'XX'
model_embs = "sentence-transformers/all-MiniLM-L6-v2"

In [ ]:
embed_model = HuggingFaceEmbedding(model_name=model_embs)
text_splitter = SentenceSplitter(chunk_size=300, chunk_overlap=90) #### need to check if this is relevant to add it after te VS is already created
service_context_with_splitter = ServiceContext.from_defaults(
        embed_model=embed_model,
        llm=llm,
        text_splitter=text_splitter
    )
# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir=os.path.join(path_project, folder_vector))

# load index
index_with_splitter = load_index_from_storage(storage_context,
                            service_context=service_context_with_splitter)

### Define template prompt

Each prompt has to be inside the function `PromptTemplate`

Prompt for retriever

In [ ]:
# Fusion Retriever

QUERY_GEN_PROMPT = (
    "You are a helpful assistant that generates multiple search queries based on a "
    "single input query. Generate {num_queries} search queries, one on each line, "
    "related to the following input query:\n"
    "Query: {query}\n"
    "Queries:\n"
)
QUERY_GEN_PROMPT_FORMATED = PromptTemplate(
    QUERY_GEN_PROMPT, prompt_type=PromptType.CUSTOM
)

Prompt for Node

In [ ]:
### LLM rerank
choice_select_prompt = ("""
A list of documents is shown below. Each document has a number next to it along with a summary of the document. A question is also provided.

Example format:
Document 1:
<summary of document 1>

Document 2:
<summary of document 2>

...

Document 10:
<summary of document 10>

In your answer, provide a score from 1 to 10, where 10 indicates fully relevant to the question, 5 indicates the document is answering the question without details and all scores below aren't very relevant.
Please score each document, and do not provide an explanation. Use the following format in your response:

[
  {{
  'document': first documents number,
  'score': first document's score
  }},
  {{
  'document': second documents number,
  'score': second document's score
  }},
  etc
]
Let's try this now:

{context_str}
Question: {query_str}
Answer:
""")
template_answer_updated = PromptTemplate(
    choice_select_prompt, prompt_type=PromptType.CHOICE_SELECT
)


Prompt for response

In [ ]:
DEFAULT_SUMMARY_PROMPT_TMPL = """Write a summary to answer the following question: {query_str}
Try to use only the information provided.
Try to include as many key details as possible
{context_str}
SUMMARY:"""

In [ ]:
# Define the function that will be called when the button is pressed
DEFAULT_TEXT_QA_PROMPT= """Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge,
answer the query.
Query: {query_str}
Answer:
"""

In [ ]:
DEFAULT_REFINE_PROMPT = """
The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer
(only if needed) with some more context below
------------
{context_msg}
------------
Given the new context, refine the original answer to better
answer the query.
If the context isn't useful, return the original answer
Refined Answer:
"""

In [ ]:
DEFAULT_TREE_SUMMARIZE_TMPL = """Context information from multiple sources is below.
---------------------
{context_str}
---------------------
Given the information from multiple sources and not prior knowledge,
answer the query.
Query: {query_str}
Answer: """

### Function utils

In [ ]:
def retrieve_nodes_for_multiple_filters(filter_dict, mode="full"):
    # Initialize an empty list to hold all retrieved nodes

    all_nodes = []
    seen_node_ids = set()

    if mode == "or":
        # Original logic: Retrieve nodes for each key-value pair separately

        for key, values in filter_dict.items():
            for value in values:
                filters = MetadataFilters(
                    filters=[ExactMatchFilter(key=key, value=value)]
                )
                query_engine = index_with_splitter.as_query_engine(
                    filters=filters, similarity_top_k=10000
                )
                #### Strategy full

                nodes = query_engine.retrieve("full")
                list_ids = [node.id_ for node in nodes]
                list_nodes = index_with_splitter.docstore.get_nodes(node_ids=list_ids)
                all_nodes.extend(list_nodes)
    elif mode == "and":
        # 'AND' logic: Retrieve nodes that match all key-value pairs simultaneously
        # Generate all combinations of key-value pairs

        all_combinations = list(product(*filter_dict.values()))

        for combination in all_combinations:
            filters = MetadataFilters(
                filters=[
                    MetadataFilter(key=key, value=value, operator=FilterOperator.EQ)
                    for key, value in zip(filter_dict.keys(), combination)
                ],
                condition=FilterCondition.AND,
            )
            query_engine = index_with_splitter.as_query_engine(
                filters=filters, similarity_top_k=10000
            )
            #### Strategy full

            nodes = query_engine.retrieve("full")
            list_ids = [node.id_ for node in nodes]
            list_nodes = index_with_splitter.docstore.get_nodes(node_ids=list_ids)
            all_nodes.extend(list_nodes)
    else:
        raise ValueError("Invalid mode. Use 'or' or 'and'.")
    # Before returning all_nodes, filter out duplicates

    unique_nodes = [
        node
        for node in all_nodes
        if node.id_ not in seen_node_ids and not seen_node_ids.add(node.id_)
    ]

    ## generate the vectorStore

    service_context_with_splitter_1 = ServiceContext.from_defaults(
        embed_model=embed_model,
        llm=llm,
        # text_splitter=text_splitter
    )
    vector_store_temp = VectorStoreIndex(
        unique_nodes, service_context=service_context_with_splitter_1
    )

    return unique_nodes, vector_store_temp


In [ ]:
def flatten_comprehension(matrix, remove_dup = True):
    if remove_dup:
        return list(set([item for row in matrix for item in row]))
    else:
        return [item for row in matrix for item in row]



In [ ]:
def load_data_from_pickle(pickle_file_name):
    with open(pickle_file_name, 'rb') as pickle_file:
        return pickle.load(pickle_file)

In [ ]:
from statistics import mean


def reconstruct_document(node_with_answer, nodes_candidates):
    ### Node response

    node_response = node_with_answer.node.id_
    nodes_candidates_ids = [i.node.id_ for i in nodes_candidates]
    ### node score
    # node_score = node_with_answer.score
    #### List best candidates

    node_parent = [
        values.node_id
        for key, values in node_with_answer.node.relationships.items()
        if key.value == "1"
    ][0]
    doc_ids = index_with_splitter.docstore.get_ref_doc_info(ref_doc_id=node_parent)
    list_nodes = index_with_splitter.docstore.get_nodes(node_ids=doc_ids.node_ids)
    list_nodes_ids = [i.id_ for i in list_nodes]
    # print(nodes_candidates, node_response)

    other_candidates = [
        i for i in list_nodes_ids if (i in nodes_candidates_ids) & (i != node_response)
    ]
    append_score = [node_with_answer.score]
    append_nodes_score = [node_with_answer]
    append_nodes_text = [node_with_answer.node.text]
    append_candidates_ids = [node_response]
    for other_candidate in other_candidates:
        r_other_candidate = [
            i for i in nodes_candidates if i.node.id_ == other_candidate
        ][0]
        append_score.append(r_other_candidate.score)
        append_nodes_score.append(r_other_candidate)
        append_nodes_text.append(r_other_candidate.text)
        append_candidates_ids.append(r_other_candidate.id_)
    #### List all nodes to use the document

    append_document_score: List[NodeWithScore] = []
    for i, node in enumerate(list_nodes):
        append_document_score.append(NodeWithScore(node=node, score=mean(append_score)))
    dic = {
        "parent": node_parent,
        "title": node_with_answer.node.metadata["title"],
        "node_best_choice": node_response,
        "node_best_choice_text": append_nodes_text,
        "list_scores": append_score,
        "nodes": doc_ids,
        "list_nodes_ids": list_nodes_ids,
        "list_nodes_candidates_ids": append_candidates_ids,
        "list_nodes": list_nodes,
        "NodeWithScore": append_nodes_score,
        "NodeWithScoreDocuments": append_document_score,
    }

    return dic


In [ ]:
def remove_duplicate_parents(list_of_dicts, key):
    unique_dicts = []
    seen_parents = set()

    for d in list_of_dicts:
        parent_value = d.get(key)  # Get the 'parent' value from the dictionary

        # Check if the 'parent' value is not in the set of seen parents

        if parent_value not in seen_parents:
            unique_dicts.append(d)  # Add the dictionary to the unique list
            seen_parents.add(
                parent_value
            )  # Add the 'parent' value to the set of seen parents
    return unique_dicts


In [ ]:
def default_parse_choice_select_answer_fn(
    answer: str, num_choices: int, raise_error: bool = False
) -> Tuple[List[int], List[float]]:
    num_choices = load_data_from_pickle("llm_score.pickle") ### temporary fix
    end_index = answer.find("]") + 1
    json_part = answer[:end_index]
    valid_json_string = json_part.replace("'", '"')

    # Parse the JSON string

    data = json.loads(valid_json_string)
    print(pd.DataFrame(data))
    data = (
        pd.DataFrame(data)
        .assign(score=lambda x: pd.to_numeric(x["score"]))
        .sort_values(by=["score"], ascending=False)
        .loc[lambda x: x["score"] >= int(num_choices)]
    )
    return list(data["document"]), list(data["score"])


### Retriver

In [ ]:
# Base Retriever
def get_base_retriever(vector_store,similarity_top_k):
    return vector_store.as_retriever(similarity_top_k=similarity_top_k) # index_with_splitter

In [ ]:
# BM25 Retriever
def get_bm25_retriever(vector_store, similarity_top_k):
    return BM25Retriever.from_defaults(docstore=vector_store.docstore, similarity_top_k=similarity_top_k)

In [ ]:
# Fusion Retriever


def get_fusion_retriever(vector_store, similarity_top_k):
    retriever_base = vector_store.as_retriever(similarity_top_k=similarity_top_k)
    retriever_bm25 = BM25Retriever.from_defaults(
        docstore=vector_store.docstore, similarity_top_k=similarity_top_k
    )
    return QueryFusionRetriever(
        [retriever_base, retriever_bm25],
        similarity_top_k=similarity_top_k,
        num_queries=4,  # set this to 1 to disable query generation
        query_gen_prompt=QUERY_GEN_PROMPT_FORMATED,
        mode="simple",  # simple
        use_async=False,
        verbose=True,
        llm=llm,
    )


### Node post

In [ ]:
def get_similarity_postprocessor(similarity_cutoff):
    return SimilarityPostprocessor(similarity_cutoff=similarity_cutoff)



In [ ]:
# Function to get the SentenceEmbeddingOptimizer
def get_sentence_embedding_optimizer(percentile_cutoff):
    return SentenceEmbeddingOptimizer(
        embed_model=service_context_with_splitter.embed_model,
        percentile_cutoff=percentile_cutoff
    )



In [ ]:
# Function to get the LLMRerank (no cutoff parameter needed)
def get_llm_rerank(top_n = 5):
    return LLMRerank(
    choice_batch_size=5,
    top_n=top_n,
    choice_select_prompt = template_answer_updated,
    parse_choice_select_answer_fn= default_parse_choice_select_answer_fn,
    service_context=service_context_with_splitter)

### Response synthetizer

In [ ]:
from llama_index.prompts import SelectorPromptTemplate

In [ ]:
def get_simple_synthesizer(prompt):
    DEFAULT_TEXT_QA_PROMPT = PromptTemplate(
    prompt, prompt_type=PromptType.SUMMARY
)
    DEFAULT_TEXT_QA_PROMPT_SEL = SelectorPromptTemplate(
    default_template=DEFAULT_TEXT_QA_PROMPT
)
    return DEFAULT_TEXT_QA_PROMPT

In [ ]:
def get_summarize_synthesizer(prompt):
    DEFAULT_TEXT_SUMMARIZE_PROMPT = PromptTemplate(
    prompt, prompt_type=PromptType.SUMMARY
)
    DEFAULT_TEXT_QA_PROMPT_SEL = SelectorPromptTemplate(
    default_template=DEFAULT_TEXT_SUMMARIZE_PROMPT
)
    return DEFAULT_TEXT_QA_PROMPT_SEL

In [ ]:
def get_refine_synthesizer(prompt):
    DEFAULT_REFINE_PROMPT_TMPL = PromptTemplate(
    prompt, prompt_type=PromptType.REFINE
)
    DEFAULT_REFINE_PROMPT_SEL = SelectorPromptTemplate(
    default_template=DEFAULT_REFINE_PROMPT_TMPL
)
    return DEFAULT_REFINE_PROMPT_SEL

In [ ]:
def get_tree_summarize_synthesizer(prompt):
    DEFAULT_TREE_SUMMARIZE_PROMPT = PromptTemplate(
    prompt, prompt_type=PromptType.SUMMARY
)
    DEFAULT_TREE_SUMMARIZE_PROMPT_SEL = SelectorPromptTemplate(
    default_template=DEFAULT_TREE_SUMMARIZE_PROMPT
)
    return DEFAULT_TREE_SUMMARIZE_PROMPT_SEL

### Generate question [optional]

In [ ]:
def generate_question(topic):
    # Retrieve the selected prompt from the dropdown
    #selected_prompt = prompt_selection#formatted_prompts[prompt_selection]
    selected_prompt = f"Can you generate only one question about the following topic: {topic}. Reply with the question only"
    # Generate the question using the LLM (you might need to adjust this part based on how you use the llm)
    question = llm.complete(selected_prompt).text

    # Return the generated question to update the text field in the interface
    return question

### Generate answer

In [ ]:
def get_response(query, synthesizer_dropdown, template_answer, nodes_or_documents):
    #print(template_answer)

    #synthesizer_qa = get_compact_summarize_synthesizer(DEFAULT_TEMPLATE_QA) ### from doc
    # https://github.com/run-llama/llama_index/blob/main/llama_index/response_synthesizers/factory.py
    # https://github.com/run-llama/llama_index/blob/main/llama_index/prompts/default_prompt_selectors.py
    # https://github.com/run-llama/llama_index/blob/main/llama_index/prompts/default_prompts.py

    if synthesizer_dropdown == "SimpleSummarize":
        response_mode = ResponseMode.SIMPLE_SUMMARIZE
        synthesizer = get_simple_synthesizer(template_answer)
        _response_synthesizer = get_response_synthesizer(
                    service_context=service_context_with_splitter,
                    callback_manager=None,
                    response_mode = response_mode,
                    text_qa_template = synthesizer
                )
    elif synthesizer_dropdown == "Summary":
        response_mode = ResponseMode.SIMPLE_SUMMARIZE
        synthesizer = get_summarize_synthesizer(template_answer)
        _response_synthesizer = get_response_synthesizer(
                    service_context=service_context_with_splitter,
                    callback_manager=None,
                    response_mode = response_mode,
                    text_qa_template=synthesizer
                )
    elif synthesizer_dropdown == "Refine":
        response_mode = ResponseMode.REFINE
        synthesizer = get_refine_synthesizer(template_answer)
        _response_synthesizer = get_response_synthesizer(
                    service_context=service_context_with_splitter,
                    callback_manager=None,
                    response_mode = response_mode,
                    refine_template  = synthesizer,
                    #text_qa_template=synthesizer_qa
                )
    elif synthesizer_dropdown == "TreeSummarize":
        response_mode = ResponseMode.TREE_SUMMARIZE
        synthesizer = get_tree_summarize_synthesizer(template_answer)
        _response_synthesizer = get_response_synthesizer(
                    service_context=service_context_with_splitter,
                    callback_manager=None,
                    response_mode = response_mode,
                    summary_template  = synthesizer,
                    #text_qa_template=synthesizer_qa
                )
    elif synthesizer_dropdown == "Compact":
        response_mode = ResponseMode.COMPACT
        synthesizer = get_refine_synthesizer(template_answer)
        _response_synthesizer = get_response_synthesizer(
                    service_context=service_context_with_splitter,
                    callback_manager=None,
                    response_mode = response_mode,
                    refine_template  = synthesizer,
                    #text_qa_template=synthesizer_qa
                )
    else:
        return "Invalid synthesizer choice."

    pickle_file_name = 'list_of_dicts.pkl'

    # Open the file in binary read mode and load the data
    loaded_list_of_dicts = load_data_from_pickle('list_of_dicts.pkl')

    print(nodes_or_documents)

    if nodes_or_documents == "Nodes":
      key_nodes = "NodeWithScore" ### bug?
      list_full_nodes_responce = flatten_comprehension([i["NodeWithScore"] for i in loaded_list_of_dicts], remove_dup = False)
    else:
      key_nodes = "NodeWithScoreDocuments"
      list_full_nodes_responce = flatten_comprehension([i[key_nodes] for i in loaded_list_of_dicts], remove_dup = False)


    print(len(list_full_nodes_responce))
    print(list_full_nodes_responce[0])

    response = _response_synthesizer.synthesize(
        query=query,
        nodes=list_full_nodes_responce
    )
    return response

### App

In [ ]:
import pickle


def updated_get_candidates(
    retriever_choice,
    similarity_top_k,
    postprocessor_choice,
    postprocessor_cutoff,
    filter_option,
    filter_keywords,
    filter_date,
    question,
):
    # Use the custom_qa_prompt to create a new PromptTemplate
    # custom_prompt_template = PromptTemplate(custom_qa_prompt, prompt_type=PromptType.QUESTION_ANSWER)

    # print(filter_option)

    if filter_keywords or filter_date:
        filter_dict = {}
        # Conditionally add each filter to the dictionary if it's not empty
        
        #### change below to adapt your meta data; add more if needed

        if filter_keywords:
            filter_dict["tags"] = filter_keywords
        if filter_date:
            filter_dict["date"] = filter_date
        unique_nodes, vector_store_temp = retrieve_nodes_for_multiple_filters(
            filter_dict, mode=filter_option
        )
        if retriever_choice == "Base":
            retriever = get_base_retriever(vector_store_temp, similarity_top_k)
        elif retriever_choice == "BM25":
            retriever = get_bm25_retriever(vector_store_temp, similarity_top_k)
        elif retriever_choice == "Fusion":
            retriever = get_fusion_retriever(vector_store_temp, similarity_top_k)
        else:
            return "Invalid retriever choice."
    else:
        # Use the selected retriever without filtering

        if retriever_choice == "Base":
            retriever = get_base_retriever(index_with_splitter, similarity_top_k)
        elif retriever_choice == "BM25":
            retriever = get_bm25_retriever(index_with_splitter, similarity_top_k)
        elif retriever_choice == "Fusion":
            retriever = get_fusion_retriever(index_with_splitter, similarity_top_k)
        else:
            return "Invalid retriever choice."
    # Retrieve the postprocessor based on user's choice

    print(postprocessor_cutoff)

    if postprocessor_choice == "Similarity":
        postprocessor = get_similarity_postprocessor(postprocessor_cutoff)
    elif postprocessor_choice == "SentenceEmbedding":
        postprocessor = get_sentence_embedding_optimizer(postprocessor_cutoff)
    elif postprocessor_choice == "LLMRerank":
        #### save locally because the value is not passed throught

        with open("llm_score.pickle", "wb") as pickle_file:
            pickle.dump(postprocessor_cutoff, pickle_file)
        postprocessor = get_llm_rerank(postprocessor_cutoff)  ### should be int
    else:
        return "Invalid postprocessor choice."
    # Retrieve the synthesizer based on user's choice

    ### Define the engine

    query_engine = RetrieverQueryEngine.from_args(
        retriever,
        service_context=service_context_with_splitter,
        node_postprocessors=(
            [postprocessor]
            if postprocessor_choice in ["Similarity", "SentenceEmbedding", "LLMRerank"]
            else None
        ),
        response_mode=ResponseMode.SIMPLE_SUMMARIZE,
    )

    #### Find documents

    results = query_engine.retrieve(QueryBundle(question))
    ### Extract all documents

    all_docs = [
        reconstruct_document(node_with_answer=i, nodes_candidates=results)
        for i in results
    ]
    all_docs_removed_dup = remove_duplicate_parents(all_docs, "parent")
    pickle_file_name = "list_of_dicts.pkl"

    # Open the file in binary write mode and save the list

    with open(pickle_file_name, "wb") as pickle_file:
        pickle.dump(all_docs_removed_dup, pickle_file)
    return len(all_docs_removed_dup)


In [ ]:
#pickle_file_name = 'list_of_dicts.pkl'
#with open(pickle_file_name, 'wb') as pickle_file:
#        pickle.dump([], pickle_file)

In [ ]:
def update_response_dropdown():
    # This function updates the dropdown choices.
    # The second parameter is not used, but necessary for the function signature.
    pickle_file_name = 'list_of_dicts.pkl'

    # Open the file in binary read mode and load the data
    loaded_list_of_dicts = load_data_from_pickle('list_of_dicts.pkl')

    return gr.Dropdown(choices=[i for i in range(len(loaded_list_of_dicts))], interactive=True)

In [ ]:
def show_data_title(index):
    pickle_file_name = 'list_of_dicts.pkl'

    # Open the file in binary read mode and load the data
    loaded_list_of_dicts = load_data_from_pickle('list_of_dicts.pkl')
    return loaded_list_of_dicts[index]['title']

def show_data_score(index):
    pickle_file_name = 'list_of_dicts.pkl'

    # Open the file in binary read mode and load the data
    loaded_list_of_dicts = load_data_from_pickle('list_of_dicts.pkl')
    return loaded_list_of_dicts[index]['list_scores']

def show_data_content(index):
    pickle_file_name = 'list_of_dicts.pkl'
    # Open the file in binary read mode and load the data
    loaded_list_of_dicts = load_data_from_pickle('list_of_dicts.pkl')
    return loaded_list_of_dicts[index]['node_best_choice_text']

In [ ]:
with gr.Blocks() as demo:
    # Define the components

    with gr.Row():
        formatted_prompts_dropdown = gr.Textbox(label="Write prompt", interactive=True)
        generate_button = gr.Button("Generate Question")
    question_display = gr.Textbox(label="Generated Question", interactive=True)

    with gr.Row():
        retriever_dropdown = gr.Dropdown(
            label="Retriever", choices=["Base", "BM25", "Fusion"], value="Base"
        )
        similarity_top_k_input = gr.Number(label="Number of Documents", value=10)
    with gr.Row():
        postprocessor_dropdown = gr.Dropdown(
            label="Postprocessor",
            choices=["Similarity", "SentenceEmbedding", "LLMRerank"],
            value="Similarity",
        )
        postprocessor_cutoff_input = gr.Number(label="Cutoff Value", value=0.8)
    ### filter
    # Multi-select dropdown for filter values

    with gr.Row():
        filter_option_metadata = gr.Dropdown(
            label="And / Or", choices=["or", "and"], value="or"
        )
        filter_keyword_dropdown = gr.Dropdown(
            label="Filter tags",
            choices=keyword,  # Update choices with your actual filter values
            # type="index",
            value=[],
            multiselect=True,  # Allow multiple selection
        )
        filter_date_dropdown = gr.Dropdown(
            label="Filter date",
            choices=date,  # Update choices with your actual filter values
            # type="index",
            value=[],
            multiselect=True,  # Allow multiple selection
        )
    with gr.Row():
        get_response_button = gr.Button("Get Response")
        display_found = gr.Textbox(label="Number of total nodes", interactive=True)
    with gr.Row():
        fetch_docs = gr.Dropdown(choices=["Wait", "update list"], label="Category")
        response_dropdown = gr.Dropdown(
            label="Responses", choices=[], value="Select a response", interactive=True
        )
    #### shwo response

    with gr.Row():
        display_title = gr.Textbox(label="Title", interactive=True)
        display_score = gr.Textbox(label="Score", interactive=True)
        display_text = gr.Textbox(label="Content", interactive=True)
    with gr.Row():
        node_document_dropdown = gr.Dropdown(
            label="Search Mode", choices=["Nodes", "Documents"], value="Nodes"
        )
        synthesizer_dropdown = gr.Dropdown(
            label="Response Synthesizer",
            choices=[
                "SimpleSummarize",
                "Summary",
                "Refine",
                "TreeSummarize",
                "Compact",
            ],
            value="SimpleSummarize",
        )
        custom_qa_prompt_input = gr.Textbox(
            label="Custom QA Prompt Template", value=DEFAULT_TEXT_QA_PROMPT
        )
    synthesizer_prompt_templates = {
        "SimpleSummarize": DEFAULT_TEXT_QA_PROMPT,  # from
        "Summary": DEFAULT_SUMMARY_PROMPT_TMPL,
        "Refine": DEFAULT_REFINE_PROMPT,
        "TreeSummarize": DEFAULT_TREE_SUMMARIZE_TMPL,  # Assuming the same template for SimpleSummarize and TreeSummarize
        "Compact": DEFAULT_REFINE_PROMPT,  # from doc
    }

    def on_synthesizer_change(synthesizer_choice):
        return synthesizer_prompt_templates[synthesizer_choice]

    synthesizer_dropdown.change(
        on_synthesizer_change,
        inputs=synthesizer_dropdown,
        outputs=custom_qa_prompt_input,
    )

    ### Generate answer

    with gr.Row():
        generate_final_answer = gr.Button("Generate answer")
        display_answer = gr.Textbox(label="Final Answer", interactive=True)
    response_dropdown.change(
        show_data_title,
        inputs=response_dropdown,
        outputs=display_title,
    )

    response_dropdown.change(
        show_data_score,
        inputs=response_dropdown,
        outputs=display_score,
    )

    response_dropdown.change(
        show_data_content,
        inputs=response_dropdown,
        outputs=display_text,
    )

    # Generate question if needed

    generate_button.click(
        generate_question,
        inputs=formatted_prompts_dropdown,
        outputs=[question_display],
    )

    fetch_docs.change(update_response_dropdown, outputs=[response_dropdown])

    #### Find documents

    get_response_button.click(
        updated_get_candidates,
        inputs=[
            retriever_dropdown,
            similarity_top_k_input,
            postprocessor_dropdown,
            postprocessor_cutoff_input,
            filter_option_metadata,
            filter_keyword_dropdown,
            filter_date_dropdown,
            question_display,
        ],
        outputs=[display_found],
    )

    generate_final_answer.click(
        get_response,
        inputs=[
            question_display,
            synthesizer_dropdown,
            custom_qa_prompt_input,
            node_document_dropdown,
        ],
        outputs=[display_answer],
    )


In [ ]:
demo.close()

What is the state of import and export in China?

In [ ]:
demo.launch(
    debug=True, 
    share = True,
    height = 700
           )